# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""12546697676072…","""../../../../da…","""49327754302568…",1.2927e19,"""92761114986569…","""13499704072840…",false
"""12557076424142…","""../../../../da…","""16966613573861…",1.4856e19,"""92761114986569…","""33028291098357…",true
"""17308229486063…","""../../../../da…","""26026464277683…",5.6458e17,"""16540348350994…","""13499704072840…",false
"""11456026641588…","""../../../../da…","""16966613573861…",4.0723e18,"""34562348621401…","""13499704072840…",false
"""16613282866682…","""../../../../da…","""49327754302568…",1.4677e19,"""34562348621401…","""63587430813675…",true
"""29558610592258…","""../../../../da…","""26026464277683…",1.6330e19,"""13745037089934…",null,false
"""13064988413373…","""../../../../da…","""26026464277683…",8.7675e18,"""34562348621401…","""63587430813675…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""15925381954398…","""../../../../da…","""11532328746764…",1.2927e19,"""FirstMortgage3…","""43851471843673…","""Rejected"""
"""83659440274125…","""../../../../da…","""47955807275153…",1.4856e19,"""FirstMortgage3…","""11873314817625…","""Accepted"""
"""13517148648945…","""../../../../da…","""61110645109259…",5.6458e17,"""MoneyMarketSav…","""43851471843673…","""Rejected"""
"""19297978356866…","""../../../../da…","""47955807275153…",4.0723e18,"""BasicChecking""","""43851471843673…","""Rejected"""
"""10147136748832…","""../../../../da…","""11532328746764…",1.4677e19,"""BasicChecking""","""58432039416933…","""Accepted"""
"""13142403649294…","""../../../../da…","""61110645109259…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""51419392069898…","""../../../../da…","""61110645109259…",8.7675e18,"""BasicChecking""","""58432039416933…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""13096620945094…","""../../../../da…","""13938191726898…",1.2927e19,"""FirstMortgage3…","""17610573532191…","""Rejected"""
"""93000384018447…","""../../../../da…","""33484828577925…",1.4856e19,"""FirstMortgage3…","""12388552480612…","""Accepted"""
"""10609713293052…","""../../../../da…","""16021220543328…",5.6458e17,"""MoneyMarketSav…","""17610573532191…","""Rejected"""
"""13047099654961…","""../../../../da…","""33484828577925…",4.0723e18,"""BasicChecking""","""17610573532191…","""Rejected"""
"""50582148829362…","""../../../../da…","""13938191726898…",1.4677e19,"""BasicChecking""","""41651629113916…","""Accepted"""
"""17862096217517…","""../../../../da…","""16021220543328…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""53661338192978…","""../../../../da…","""16021220543328…",8.7675e18,"""BasicChecking""","""41651629113916…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""87760511467100…","""82138307069491…",1.2927e19,"""FirstMortgage3…","""94021825356432…","""Rejected"""
"""10402501295592…","""96689285444426…",1.4856e19,"""FirstMortgage3…","""19684556888713…","""Accepted"""
"""14378711884404…","""66688036769354…",5.6458e17,"""MoneyMarketSav…","""94021825356432…","""Rejected"""
"""43891404852843…","""96689285444426…",4.0723e18,"""BasicChecking""","""94021825356432…","""Rejected"""
"""49471672254077…","""82138307069491…",1.4677e19,"""BasicChecking""","""36513111754913…","""Accepted"""
"""82710483972204…","""66688036769354…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""81627746272986…","""66688036769354…",8.7675e18,"""BasicChecking""","""36513111754913…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'